In [1]:
#---Cloud search libraries
import s3fs
import requests
import fnmatch

#---Accessory libraries
import datetime
import netCDF4

**Set the datetime and band to view:**

In [2]:
year = 2024
month = 6
day = 20
hour = 6

band = '14' #---Include the leading zero, if needed

julian_day = datetime.datetime(year, month, day).strftime('%j')
print(julian_day)

datetime_str = str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2)+' '+str(hour).zfill(2)+'Z'

172


**Connecting to AWS remote storage:**

In [3]:
fs = s3fs.S3FileSystem(anon=True)

**Search the AWS database:**

In [4]:
#---NOAA-20
bucket = 'noaa-nesdis-n20-pds'

files = fs.ls(bucket+'/'+'VIIRS-DNB-SDR/'+str(year)+'/'+str(month).zfill(2)+'/'+str(day).zfill(2)+'/')
files_geo = fs.ls(bucket+'/'+'VIIRS-DNB-GEO/'+str(year)+'/'+str(month).zfill(2)+'/'+str(day).zfill(2)+'/')

files_hour = [file for file in files if fnmatch.fnmatch(file.split('/')[-1], 'SVDNB_j01_d'+str(year)+str(month).zfill(2)+str(day)+'_t'+str(hour).zfill(2)+'*')]
files_hour_geo = [file for file in files_geo if fnmatch.fnmatch(file.split('/')[-1], 'GDNBO_j01_d'+str(year)+str(month).zfill(2)+str(day)+'_t'+str(hour).zfill(2)+'*')]

print(files_hour[0])
print(files_hour_geo[0])

noaa-nesdis-n20-pds/VIIRS-DNB-SDR/2024/06/20/SVDNB_j01_d20240620_t0600123_e0601350_b34134_c20240620061221033000_oebc_ops.h5
noaa-nesdis-n20-pds/VIIRS-DNB-GEO/2024/06/20/GDNBO_j01_d20240620_t0600123_e0601350_b34134_c20240620061023832000_oebc_ops.h5


In [5]:
file = files_hour[0]
file_geo = files_hour_geo[0]

resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+file[20:])
if str(resp) != '<Response [200]>':
    print('DNB file not found in AWS servers')

resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+file_geo[20:])
if str(resp) != '<Response [200]>':
    print('DNB GEO file not found in AWS servers')



nc = netCDF4.Dataset(file, memory = resp.content)
nc_geo = netCDF4.Dataset(file_geo, memory = resp.content)

In [6]:
nc.groups['All_Data']['VIIRS-DNB-GEO_All']

<class 'netCDF4._netCDF4.Group'>
group /All_Data/VIIRS-DNB-GEO_All:
    dimensions(sizes): phony_dim_0(768), phony_dim_1(4064), phony_dim_2(48), phony_dim_3(1), phony_dim_4(3)
    variables(dimensions): float32 Height(phony_dim_0, phony_dim_1), float32 Height_TC(phony_dim_0, phony_dim_1), float32 Latitude(phony_dim_0, phony_dim_1), float32 Latitude_TC(phony_dim_0, phony_dim_1), float32 Longitude(phony_dim_0, phony_dim_1), float32 Longitude_TC(phony_dim_0, phony_dim_1), float32 LunarAzimuthAngle(phony_dim_0, phony_dim_1), float32 LunarZenithAngle(phony_dim_0, phony_dim_1), int64 MidTime(phony_dim_2), uint8 ModeGran(phony_dim_3), uint8 ModeScan(phony_dim_2), float32 MoonIllumFraction(phony_dim_3), float32 MoonPhaseAngle(phony_dim_3), int32 NumberOfScans(phony_dim_3), uint8 PadByte1(phony_dim_4), uint8 QF1_SCAN_VIIRSSDRGEO(phony_dim_2), uint8 QF2_SCAN_VIIRSSDRGEO(phony_dim_2), uint8 QF2_VIIRSSDRGEO(phony_dim_0, phony_dim_1), uint8 QF2_VIIRSSDRGEO_TC(phony_dim_0, phony_dim_1), float32 SCAt